# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('../../open_ai.env')) # read the env file from parent directory

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
You are an expert SQL assistant. Generate valid SQLite queries based on user questions.
Always provide clear, efficient SQL commands followed by a brief explanation.

Database Schema:

-- Table: employees
CREATE TABLE employees (
    ID_usr INTEGER PRIMARY KEY,
    name VARCHAR(100) NOT NULL
);

-- Table: salary
CREATE TABLE salary (
    ID_usr INTEGER NOT NULL,
    year DATE NOT NULL,
    salary FLOAT NOT NULL,
    PRIMARY KEY (ID_usr, year),
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

-- Table: studies
CREATE TABLE studies (
    ID INTEGER PRIMARY KEY,
    ID_usr INTEGER NOT NULL,
    educational_level INTEGER,
    Institution VARCHAR(200),
    Years DATE,
    Speciality VARCHAR(100),
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

-- Sample Data:
-- employees: ID_usr, name
-- 1, 'John Smith'
-- 2, 'Maria Garcia'
-- 3, 'James Johnson'

-- salary: ID_usr, year, salary
-- 1, '2023-01-01', 75000.00
-- 2, '2023-01-01', 82000.00
-- 3, '2023-01-01', 68000.00

-- studies: ID, ID_usr, educational_level, Institution, Years, Speciality
-- 1, 1, 4, 'MIT', '2015-01-01', 'Computer Science'
-- 2, 2, 5, 'Stanford', '2016-01-01', 'Data Science'
-- 3, 3, 3, 'UC Berkeley', '2018-01-01', 'Business Administration'
"""} ]

In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
-- Maintain the SQL simple and efficient using valid SQLite syntax.

-- Example 1:
-- Question: Show me all employee names
-- SQL:
SELECT name FROM employees;

-- Example 2:
-- Question: What is the average salary in 2023?
-- SQL:
SELECT AVG(salary) as average_salary 
FROM salary 
WHERE year = '2023-01-01';

-- Example 3:
-- Question: List employees with their salaries for 2023
-- SQL:
SELECT e.name, s.salary 
FROM employees e 
JOIN salary s ON e.ID_usr = s.ID_usr 
WHERE s.year = '2023-01-01';

-- Example 4:
-- Question: Which employees have a Master's degree or higher (educational_level >= 5)?
-- SQL:
SELECT DISTINCT e.name, st.educational_level, st.Institution 
FROM employees e 
JOIN studies st ON e.ID_usr = st.ID_usr 
WHERE st.educational_level >= 5;

-- Example 5:
-- Question: Show employees with their education and current salary
-- SQL:
SELECT e.name, st.Institution, st.Speciality, s.salary 
FROM employees e 
LEFT JOIN studies st ON e.ID_usr = st.ID_usr 
LEFT JOIN salary s ON e.ID_usr = s.ID_usr 
WHERE s.year = '2023-01-01';
"""
})

In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [14]:
q1 = 'Are employees with advanced degrees promoted or compensated faster than others?'
q2 = "Which employees received salary increases after completing additional studies?"


In [12]:
#new
context_user = context.copy()
print(return_CCRMSQL(q1, context_user))

SQL:
```sql
SELECT e.name, st.educational_level, s.salary 
FROM employees e 
LEFT JOIN studies st ON e.ID_usr = st.ID_usr 
LEFT JOIN salary s ON e.ID_usr = s.ID_usr 
WHERE s.year = '2023-01-01' 
ORDER BY st.educational_level DESC, s.salary DESC;
```

Explanation:
This query retrieves the names of employees along with their educational levels and salaries for the year 2023. It uses LEFT JOINs to include all employees, even those without studies or salary records. The results are ordered by educational level (higher degrees first) and salary (higher salaries first), which can help analyze if employees with advanced degrees are compensated better.


In [13]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL(q1, old_context_user))

That's an interesting question! Please ask something SQL-related.


In [15]:
#new
print(return_CCRMSQL(q2, context_user))

```sql
SELECT e.name, s1.salary AS previous_salary, s2.salary AS current_salary, st.Institution, st.Speciality
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
JOIN salary s1 ON e.ID_usr = s1.ID_usr AND s1.year < st.Years
JOIN salary s2 ON e.ID_usr = s2.ID_usr AND s2.year = '2023-01-01'
WHERE s2.salary > s1.salary;
```

This query retrieves the names of employees who received salary increases after completing additional studies. It joins the `employees`, `studies`, and `salary` tables to compare the salary before and after the completion of their studies, ensuring that the current salary is greater than the previous salary.


In [16]:
#old
print(return_CCRMSQL(q2, old_context_user))

This is your SQL:

```sql
SELECT e.name, s.salary, st.educational_level
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE st.educational_level > (SELECT MAX(educational_level) FROM studies WHERE ID_usr = e.ID_usr);
```

This query retrieves employees who have completed additional studies and checks if their educational level is higher than their previous level, indicating a potential salary increase.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

Key Learnings from the Text-to-SQL Research:
1. Schema Representation Matters

Normalized CREATE TABLE syntax outperforms JSON or other formats
Standard SQL syntax reduces token count and improves accuracy
The model understands database structure better when presented in familiar SQL format
2. Relationships Are Critical

Foreign keys significantly improve performance (71.8% → 73.1% for Codex)
Explicitly defining table relationships helps the model generate correct JOINs
Without foreign keys, models struggle to understand which tables connect and how
3. Sample Data Is Essential

Including sample rows helps models understand data types and content
SelectCol format (column-first listing) works better than INSERT statements
Sample data prevents hallucination of column names or incorrect value assumptions
Models can infer data patterns (e.g., date formats, naming conventions)
4. Few-Shot Examples Are Powerful

Providing 3-5 example query pairs dramatically improves accuracy
Examples teach the model:
Your preferred SQL style and syntax
How to handle JOINs properly
Common query patterns for your schema
Edge cases and best practices
5. Consistency Prevents Errors

Fixed inconsistencies (nombre vs name, tablename vs tableName) reduce confusion
Specifying SQLite dialect explicitly prevents syntax errors
Clear naming conventions help the model generate more accurate queries
Practical Impact:
The old prompt used JSON format without foreign keys or examples, which likely caused the model to:

Generate incorrect JOIN conditions
Hallucinate column names (mixing "nombre" and "name")
Use wrong SQL dialect syntax
Miss obvious table relationships
